In [159]:
import pandas as pd

In [160]:
import os
gpx_files_all = os.listdir('data/ride/')
gpx_files = []
for each in gpx_files_all:
    #if 'timestamp_updated' in each:
        gpx_files.append('data/ride/' + each)

gpx_files

['data/ride/8999076307.gpx',
 'data/ride/8932015947.gpx',
 'data/ride/9115934547.gpx',
 'data/ride/9046182374.gpx',
 'data/ride/8914248054.gpx',
 'data/ride/8997155702.gpx',
 'data/ride/8987516473.gpx',
 'data/ride/9084726470.gpx',
 'data/ride/8987607129.gpx',
 'data/ride/8993016108.gpx',
 'data/ride/9006525740.gpx',
 'data/ride/9003173105.gpx',
 'data/ride/8885774506.gpx',
 'data/ride/8950086580.gpx',
 'data/ride/8967826254.gpx',
 'data/ride/8926392748.gpx',
 'data/ride/8919279228.gpx',
 'data/ride/9077350457.gpx',
 'data/ride/8991011278.gpx',
 'data/ride/8939029175.gpx',
 'data/ride/9075943483.gpx',
 'data/ride/8907088806.gpx',
 'data/ride/8890859129.gpx',
 'data/ride/9058117223.gpx',
 'data/ride/9091336682.gpx',
 'data/ride/9012483697.gpx',
 'data/ride/8957935093.gpx',
 'data/ride/9066438551.gpx',
 'data/ride/9012436563.gpx',
 'data/ride/8955805139.gpx',
 'data/ride/9035575071.gpx',
 'data/ride/9088274244.gpx',
 'data/ride/8975088974.gpx',
 'data/ride/9043247633.gpx',
 'data/ride/91

In [100]:
from xml.etree import ElementTree as ET

# Register the namespace for GPX files
ET.register_namespace("", "http://www.topografix.com/GPX/1/1")

def extract_start_end_points(gpx_file):
    # Parse the GPX file
    tree = ET.parse(gpx_file)
    root = tree.getroot()

    # Get all trackpoints
    trkpts = root.findall(".//{http://www.topografix.com/GPX/1/1}trkpt")

    # Extract coordinates for the first (start) and last (end) trackpoint
    start_point = (float(trkpts[0].attrib['lat']), float(trkpts[0].attrib['lon']))
    end_point = (float(trkpts[-1].attrib['lat']), float(trkpts[-1].attrib['lon']))

    return start_point, end_point

# List to store start and end points for all activities
start_end_points = []


In [101]:
# For each GPX file, extract start and end points and append to the list
for gpx_file in gpx_files:
    start_end_points.append(extract_start_end_points(gpx_file))

print(start_end_points)

[((33.64753, -117.839787), (33.668904, -117.827405)), ((33.669144, -117.827954), (33.644806, -117.841554)), ((33.669056, -117.827401), (33.643305, -117.84317)), ((33.644008, -117.840887), (33.668993, -117.827413)), ((33.645414, -117.84126), (33.668798, -117.827242)), ((33.644946, -117.840311), (33.66901, -117.827399)), ((33.66919, -117.827985), (33.644808, -117.841607)), ((33.645115, -117.841553), (33.668859, -117.827332)), ((33.669295, -117.827659), (33.644833, -117.841617)), ((33.669268, -117.828006), (33.644848, -117.841654)), ((33.669359, -117.827648), (33.644857, -117.841624)), ((33.649514, -117.839495), (33.668913, -117.827377)), ((33.657136, -117.827977), (33.643187, -117.843242)), ((33.669091, -117.827321), (33.668928, -117.827358)), ((33.667865, -117.827521), (33.657616, -117.827958)), ((33.644774, -117.841593), (33.668856, -117.827368)), ((33.669131, -117.827962), (33.644817, -117.841627)), ((33.64992, -117.83831), (33.668893, -117.827373)), ((33.650335, -117.840679), (33.668

In [102]:
start_end_points

[((33.64753, -117.839787), (33.668904, -117.827405)),
 ((33.669144, -117.827954), (33.644806, -117.841554)),
 ((33.669056, -117.827401), (33.643305, -117.84317)),
 ((33.644008, -117.840887), (33.668993, -117.827413)),
 ((33.645414, -117.84126), (33.668798, -117.827242)),
 ((33.644946, -117.840311), (33.66901, -117.827399)),
 ((33.66919, -117.827985), (33.644808, -117.841607)),
 ((33.645115, -117.841553), (33.668859, -117.827332)),
 ((33.669295, -117.827659), (33.644833, -117.841617)),
 ((33.669268, -117.828006), (33.644848, -117.841654)),
 ((33.669359, -117.827648), (33.644857, -117.841624)),
 ((33.649514, -117.839495), (33.668913, -117.827377)),
 ((33.657136, -117.827977), (33.643187, -117.843242)),
 ((33.669091, -117.827321), (33.668928, -117.827358)),
 ((33.667865, -117.827521), (33.657616, -117.827958)),
 ((33.644774, -117.841593), (33.668856, -117.827368)),
 ((33.669131, -117.827962), (33.644817, -117.841627)),
 ((33.64992, -117.83831), (33.668893, -117.827373)),
 ((33.650335, -11

In [107]:
from sklearn.cluster import DBSCAN
import numpy as np

points = [point for start_end in start_end_points for point in start_end]

X = np.array(points)

eps_in_radians = 0.25 / 6371.0088  # this is approximately 500 meters in radians
min_samples = 4  # adjust this number based on how many activities need to share a location for it to be considered common

db = DBSCAN(eps=eps_in_radians, min_samples=min_samples, metric='haversine').fit(np.radians(X))

labels = db.labels_

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

print('Cluster count: %d' % n_clusters)


Cluster count: 2


In [108]:
import folium

# Create a map centered at the mean of the points
map_center = [np.mean(X[:, 0]), np.mean(X[:, 1])]
m = folium.Map(location=map_center, zoom_start=13)

# Create a color dictionary for each cluster ID
colors = {i: '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) for i in range(n_clusters)}

# add a marker for every record in the filtered data, use a clustered view
for idx, coordinates in enumerate(X):
    cluster_id = labels[idx]
    if cluster_id != -1:  # Exclude the noise points
        folium.CircleMarker(
            location=coordinates,
            fill=True,
            radius=5,
            weight=1,
            color=colors[cluster_id],
            fill_color=colors[cluster_id],
            fill_opacity=1
        ).add_to(m)

m.save('cluster_map.html')


In [109]:
points

[(33.64753, -117.839787),
 (33.668904, -117.827405),
 (33.669144, -117.827954),
 (33.644806, -117.841554),
 (33.669056, -117.827401),
 (33.643305, -117.84317),
 (33.644008, -117.840887),
 (33.668993, -117.827413),
 (33.645414, -117.84126),
 (33.668798, -117.827242),
 (33.644946, -117.840311),
 (33.66901, -117.827399),
 (33.66919, -117.827985),
 (33.644808, -117.841607),
 (33.645115, -117.841553),
 (33.668859, -117.827332),
 (33.669295, -117.827659),
 (33.644833, -117.841617),
 (33.669268, -117.828006),
 (33.644848, -117.841654),
 (33.669359, -117.827648),
 (33.644857, -117.841624),
 (33.649514, -117.839495),
 (33.668913, -117.827377),
 (33.657136, -117.827977),
 (33.643187, -117.843242),
 (33.669091, -117.827321),
 (33.668928, -117.827358),
 (33.667865, -117.827521),
 (33.657616, -117.827958),
 (33.644774, -117.841593),
 (33.668856, -117.827368),
 (33.669131, -117.827962),
 (33.644817, -117.841627),
 (33.64992, -117.83831),
 (33.668893, -117.827373),
 (33.650335, -117.840679),
 (33.668

In [110]:
m

In [51]:
from xml.etree import ElementTree as ET
from datetime import datetime
import os

def get_start_end_points_timestamps(gpx_folder):
    start_end_points_timestamps = []

    for gpx_file in os.listdir(gpx_folder):
        if gpx_file.endswith(".gpx"):
            tree = ET.parse(os.path.join(gpx_folder, gpx_file))
            root = tree.getroot()

            # Extract all trackpoints (trkpt elements)
            trackpoints = root.findall('.//{http://www.topografix.com/GPX/1/1}trkpt')
            if trackpoints:
                # Get the first (start) and last (end) trackpoint
                start_point, end_point = trackpoints[0], trackpoints[-1]

                # Extract lat, lon and time from start and end points
                start_lat, start_lon = float(start_point.attrib['lat']), float(start_point.attrib['lon'])
                end_lat, end_lon = float(end_point.attrib['lat']), float(end_point.attrib['lon'])

                start_time_text = start_point.find('{http://www.topografix.com/GPX/1/1}time').text
                end_time_text = end_point.find('{http://www.topografix.com/GPX/1/1}time').text

                start_time = datetime.fromisoformat(start_time_text.replace('Z', '+00:00'))
                end_time = datetime.fromisoformat(end_time_text.replace('Z', '+00:00'))

                # Append the start and end coordinates and corresponding timestamps to the list
                start_end_points_timestamps.append((start_lat, start_lon, start_time, end_lat, end_lon, end_time))

    return start_end_points_timestamps

gpx_folder = "data/run/"
start_end_points_timestamps = get_start_end_points_timestamps(gpx_folder)


In [52]:
start_end_points_timestamps

[(33.643497,
  -117.843057,
  datetime.datetime(2023, 5, 5, 0, 57, 45, tzinfo=datetime.timezone.utc),
  33.642964,
  -117.844376,
  datetime.datetime(2023, 5, 5, 2, 10, 59, tzinfo=datetime.timezone.utc)),
 (33.668187,
  -117.827028,
  datetime.datetime(2023, 5, 17, 2, 41, 50, tzinfo=datetime.timezone.utc),
  33.6692,
  -117.82781,
  datetime.datetime(2023, 5, 17, 3, 34, 8, tzinfo=datetime.timezone.utc)),
 (33.668103,
  -117.827115,
  datetime.datetime(2023, 5, 6, 2, 52, 40, tzinfo=datetime.timezone.utc),
  33.67046,
  -117.830735,
  datetime.datetime(2023, 5, 6, 3, 40, 51, tzinfo=datetime.timezone.utc)),
 (33.668242,
  -117.826983,
  datetime.datetime(2023, 5, 10, 2, 16, 51, tzinfo=datetime.timezone.utc),
  33.668434,
  -117.827186,
  datetime.datetime(2023, 5, 10, 3, 21, 15, tzinfo=datetime.timezone.utc)),
 (33.668252,
  -117.826971,
  datetime.datetime(2023, 5, 11, 14, 59, 8, tzinfo=datetime.timezone.utc),
  33.668828,
  -117.827632,
  datetime.datetime(2023, 5, 11, 16, 9, 17, tzinfo

In [54]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Separate coordinates and timestamps
coordinates = [(start_lat, start_lon, end_lat, end_lon) for start_lat, start_lon, start_time, end_lat, end_lon, end_time in start_end_points_timestamps]
timestamps = [(start_time.timestamp(), end_time.timestamp()) for start_lat, start_lon, start_time, end_lat, end_lon, end_time in start_end_points_timestamps]

# Flatten lists
coordinates = [coordinate for sublist in coordinates for coordinate in sublist]
timestamps = [timestamp for sublist in timestamps for timestamp in sublist]

# Reshape into 2D arrays
coordinates = np.array(coordinates).reshape(-1, 2)
timestamps = np.array(timestamps).reshape(-1, 1)

# Scale timestamps
scaler = StandardScaler()
timestamps_scaled = scaler.fit_transform(timestamps)

# Merge back into one array
X_scaled = np.column_stack((coordinates, timestamps_scaled))


In [55]:
# Define the parameters for DBSCAN
eps_in_radians = 0.5 / 6371.0088  # this is approximately 500 meters in radians
min_samples = 2  # adjust this number based on how many activities need to share a location for it to be considered common

# Run DBSCAN
db = DBSCAN(eps=eps_in_radians, min_samples=min_samples).fit(X_scaled) # Note we are not specifying the metric, so Euclidean distance will be used

# Get the cluster labels
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters)


Estimated number of clusters: 0


In [57]:
timestamps

array([[1.68324826e+09],
       [1.68325266e+09],
       [1.68429131e+09],
       [1.68429445e+09],
       [1.68334156e+09],
       [1.68334445e+09],
       [1.68368501e+09],
       [1.68368888e+09],
       [1.68381715e+09],
       [1.68382136e+09],
       [1.68398868e+09],
       [1.68399589e+09],
       [1.68454826e+09],
       [1.68455266e+09],
       [1.68437842e+09],
       [1.68438263e+09],
       [1.68350768e+09],
       [1.68351175e+09]])

In [189]:
import gpxpy
import pandas as pd
import os
from datetime import datetime
import pytz

gpx_directory = 'data/run'
local_tz = pytz.timezone('America/Los_Angeles')

data = []
for filename in os.listdir(gpx_directory):
    if filename.endswith('.gpx'):
        gpx_file = open(os.path.join(gpx_directory, filename), 'r')
        gpx = gpxpy.parse(gpx_file)

        for track in gpx.tracks:
            for segment in track.segments:
                # Only take the first and last point of each segment
                start_point = segment.points[0]
                end_point = segment.points[-1]
                
                for point_type, point in zip(['Start', 'End'], [start_point, end_point]):
                    utc_time = datetime.utcfromtimestamp(point.time.timestamp())
                    local_time = utc_time.replace(tzinfo=pytz.utc).astimezone(local_tz)
                    hour_of_day = local_time.hour
                    data.append([point.latitude, point.longitude, hour_of_day, point_type])

df = pd.DataFrame(data, columns=['Latitude', 'Longitude', 'HourOfDay', 'PointType'])


In [190]:
df['PointFeature'] = df['PointType'].apply(lambda x: -1 if x == 'Start' else 2)

In [191]:
df['PointType'].apply(lambda x: -1 if x == 'Start' else 1)

0    -1
1     1
2    -1
3     1
4    -1
5     1
6    -1
7     1
8    -1
9     1
10   -1
11    1
12   -1
13    1
14   -1
15    1
16   -1
17    1
18   -1
19    1
20   -1
21    1
22   -1
23    1
24   -1
25    1
26   -1
27    1
28   -1
29    1
30   -1
31    1
Name: PointType, dtype: int64

In [192]:
# Normalize the features
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df[['Latitude', 'Longitude', 'HourOfDay','PointFeature']])

# Apply DBSCAN
dbscan = DBSCAN(eps=0.25, min_samples=5)  # You might want to adjust these parameters
clusters = dbscan.fit_predict(data_scaled)

# Add cluster information back to DataFrame
df['Cluster'] = clusters


In [193]:
clusters

array([-1, -1,  1,  0,  1,  0, -1, -1,  1,  0,  1,  0,  1,  0, -1, -1, -1,
       -1,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0,  1,  0])

In [194]:
import folium
import geopy.distance

# Create map centered on mean coordinates
map_center = [df['Latitude'].mean(), df['Longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=13)

# Use color palette to distinguish different clusters
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
          'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
          'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
          'gray', 'black', 'lightgray']

# For each cluster, add a feature group to the map, add a circle for each point
for cluster_id in df['Cluster'].unique():
    fg = folium.FeatureGroup(name=f'Cluster {cluster_id}')
    cluster_data = df[df['Cluster'] == cluster_id]
    
    # Calculate median 'HourOfDay' for this cluster
    median_hour = cluster_data['HourOfDay'].median()
    
    for idx, row in cluster_data.iterrows():
        folium.CircleMarker(location=[row['Latitude'], row['Longitude']],
                            radius=5,
                            fill=True,
                            color=colors[cluster_id % len(colors)],
                            fill_opacity=0.7).add_to(fg)
    
    # Calculate mean location of the cluster and maximum distance from mean location
    #mean_location = [cluster_data['Latitude'].mean(), cluster_data['Longitude'].mean()]
    #max_distance = cluster_data.apply(lambda row: geopy.distance.distance(mean_location, [row['Latitude'], row['Longitude']]).m, axis=1).max()
    
    mean_location = [cluster_data['Latitude'].mean(), cluster_data['Longitude'].mean()]
    distances = cluster_data.apply(lambda row: geopy.distance.distance(mean_location, [row['Latitude'], row['Longitude']]).m, axis=1)
    percentile_distance = distances.quantile(0.9)

    # Draw a circle around the cluster
    folium.Circle(
        location=mean_location,
        radius=percentile_distance,
        color=colors[cluster_id % len(colors)],
        fill=True,
        fill_opacity=0.2,
    ).add_to(fg)
    
    # Add separate markers for start and end points
    for point_type in ['Start', 'End']:
        point_type_data = cluster_data[cluster_data['PointType'] == point_type]
        mean_latitude = point_type_data['Latitude'].mean()
        mean_longitude = point_type_data['Longitude'].mean()
        mean_hour_of_day = point_type_data['HourOfDay'].mean()
        # Check if mean_latitude or mean_longitude is NaN before plotting
        if not pd.isna(mean_latitude) and not pd.isna(mean_longitude):
            folium.Marker(
                location=[mean_latitude, mean_longitude],
                popup=f'{point_type} Point<br>Lat: {mean_latitude:.4f}<br>Lon: {mean_longitude:.4f}<br>Hour of Day: {mean_hour_of_day:.2f}',
                icon=folium.Icon(icon="map-marker", prefix='fa')  # Use a marker icon
            ).add_to(fg)
    
    fg.add_to(m)

folium.LayerControl().add_to(m)
m.save('cluster_map.html')


In [195]:
m

In [168]:
# Count 'Start' and 'End' points separately
start_points = df[df['PointType'] == 'Start'].groupby('Cluster')['PointType'].count()
end_points = df[df['PointType'] == 'End'].groupby('Cluster')['PointType'].count()

# Calculate other statistics as before
cluster_details = df.groupby('Cluster').agg(
    MeanLatitude=('Latitude', 'mean'),
    MeanLongitude=('Longitude', 'mean'),
    MedianHourOfDay=('HourOfDay', 'median'),
    NumPoints=('Cluster', 'count')
)

# Add 'Start' and 'End' counts to the dataframe
cluster_details['StartPoints'] = start_points
cluster_details['EndPoints'] = end_points

print(cluster_details)


         MeanLatitude  MeanLongitude  MedianHourOfDay  NumPoints  StartPoints  \
Cluster                                                                         
-1          33.659355    -117.828689             14.0         10          6.0   
 0          33.646301    -117.840731             13.0         21         21.0   
 1          33.668901    -117.827354             15.5         30          NaN   
 2          33.669195    -117.827807              7.0         28         28.0   
 3          33.645242    -117.841242              8.0         26          NaN   
 4          33.668134    -117.827671             18.0          5          5.0   
 5          33.643593    -117.842239             16.5          2          NaN   
 6          33.647416    -117.831568             18.0          2          2.0   

         EndPoints  
Cluster             
-1             4.0  
 0             NaN  
 1            30.0  
 2             NaN  
 3            26.0  
 4             NaN  
 5             2.0  


In [169]:
df.columns

Index(['Latitude', 'Longitude', 'HourOfDay', 'PointType', 'PointFeature',
       'Cluster'],
      dtype='object')